In [1]:
import qiskit
import numpy as np
import random
import matplotlib.pyplot as plt
import json
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import XGate, ZGate
from qiskit import QuantumCircuit
from qiskit_aer.noise import (
    NoiseModel,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
    phase_damping_error,
    ReadoutError,
    RelaxationNoisePass
)
from qiskit_ibm_runtime import QiskitRuntimeService

In [2]:
# Save and load your IBM Quantum account credentials
'''QiskitRuntimeService.save_account(token='2fdf9a0789b4a59bc40442c862e78c1bc79eea8668e6fd2fbcf8218db84742b0d2e16f9f21d6ee4ec0a187493e52637555e892bc7e5184b787d84e844bff7ba2', channel='ibm_quantum', overwrite=True)'''
service = QiskitRuntimeService()

In [3]:
backend = service.backend('ibm_fez')

In [4]:
backend_prop = backend.properties().__dict__

In [ ]:
gate_props = backend_prop["_gates"]
gate_error_list = []
for gate_type in gate_props.keys():
    gate_qubits = list(gate_props[gate_type].keys())
    for q in gate_qubits:
        gate_error = gate_props[gate_type][q].get('gate_error', (0,0))[0]
        gate_error_list.append((gate_type, q, gate_error))
gate_error_list

In [ ]:
noise_model = NoiseModel()
for val in gate_error_list:
    gate, qubits, error = val
    dep_error = depolarizing_error(error, len(qubits) )
    noise_model.add_quantum_error(dep_error, [gate], qubits)
print(noise_model)

In [ ]:
qubit_props = backend_prop["_qubits"]
qubit_error_list = []
for qubit in qubit_props.keys():
    t1 = qubit_props[qubit]['T1'][0]
    t2 = qubit_props[qubit]['T2'][0]
    qubit_error_list.append((qubit, t1, t2))
qubit_error_list

In [8]:
l1 = 1
l2 = 1
t1s =[(1/l1)*x[1] for x in qubit_error_list]
t2s =[(1/l2)*x[2] for x in qubit_error_list]

In [9]:
delay_pass = RelaxationNoisePass(t1s, t2s, dt=1e-9, op_types=[qiskit.circuit.Delay])

In [10]:
shots = 10000
ideal_simulator = AerSimulator(method='matrix_product_state')
ideal_simulator.set_options(shots=shots)
backend_sim = AerSimulator(method='matrix_product_state',
                        noise_model=noise_model)
backend_sim.set_options(shots=shots)
basis_gates = ['cz', 'id', 'x', 'sx', 'rz']

In [11]:
def initialize_superposition_from_bitstrings(bitstrings):
    n_qubits = len(bitstrings[0])
    qc = QuantumCircuit(n_qubits)

    # 중첩 상태에 들어갈 각 상태에 대한 진폭 계산
    state = np.zeros(2**n_qubits, dtype=complex)
    
    # 각 비트열을 2진수에서 정수로 변환하여 해당 상태에 진폭 할당
    for bitstring in bitstrings:
        index = int(bitstring, 2)  # 비트열을 정수로 변환
        state[index] = 1  # 각 상태에 진폭을 1로 설정 (동등한 중첩)

    # 진폭들의 노름이 1이 되도록 정규화
    state /= np.linalg.norm(state)

    # 계산된 중첩 상태로 큐비트 초기화
    qc.initialize(state)
    tqc = transpile(qc, basis_gates=basis_gates, optimization_level=3)
    new_qc = QuantumCircuit(n_qubits)
    for gate in tqc:
        if gate.operation.name == 'reset':
            continue
        new_qc.append(gate)

    return new_qc

def find_always_zero_qubits(bitstrings):
    if not bitstrings:
        return []

    num_qubits = len(bitstrings[0])  # 비트스트링의 길이(큐빗 수)
    always_zero_qubits = []

    for i in range(num_qubits):
        # i번째 위치의 큐빗이 항상 0인지 확인
        if all(bitstring[num_qubits-1-i] == '0' for bitstring in bitstrings):
            always_zero_qubits.append(i)

    return always_zero_qubits

def extract_non_zero_bitstrings(psi, total_qubits):
    bitstrings = []
    num_states = len(psi)  # 전체 상태 벡터의 길이
    
    for i, amplitude in enumerate(psi):
        if np.abs(amplitude) > 1e-10:  # 무시할 수 없는 크기의 상태만 고려 1e-12
            binary_state = f"{i:0{total_qubits}b}"
            bitstrings.append(binary_state)
    
    return bitstrings

def err_grover(n, search_bs, targ_bs):
    if len(search_bs) == 2**n:
        qc_init = QuantumCircuit(n)
        for i in range(n):
            qc_init.h(i)
    else:
        qc_init = initialize_superposition_from_bitstrings(search_bs)
    qc_oracle = QuantumCircuit(n+1)
    for tbs in targ_bs:
        qc_oracle.append(XGate().control(n, ctrl_state=tbs), range(n+1))
    
    qc = QuantumCircuit(n+1, n)
    qc.compose(qc_init, inplace=True)
    qc.x(n)
    qc.h(n)
    
    qc.compose(qc_oracle, inplace=True)
    
    qc.compose(qc_init.inverse(), inplace=True)
    #qc.save_statevector(f'psi')
    tqc = transpile(qc, basis_gates=basis_gates, optimization_level=3)
    for i in range(n):
        tqc.measure(i, i)
    # Run simulator
    result = backend_sim.run(tqc).result()
    counts = result.get_counts(0)
    #print(counts)
    
    #psi = np.array(result.data()[f'psi'])
    #total_qubits = n+1
    #bitstrings_psi = extract_non_zero_bitstrings(psi, total_qubits)
    bitstring_reduce = {}
    max_count = np.max(list(counts.values()))
    for key, count in counts.items():
        if count > max_count/10:
            bitstring_reduce[key] = count
    #print(bitstring_reduce)
    
    always_zero = find_always_zero_qubits(list(bitstring_reduce.keys()))
    #print(always_zero)
    
    for i in range(n):
        qc.x(i)
    
    q_list = list(range(n))
    #print("indices of zero qubits", always_zero)
    # Select the target qubit as the highest-indexed qubit among the remaining qubits
    # outside of oracle range (4) excluded
    q_list = [q for q in q_list if q not in always_zero and q < n]
    #print("reduced diff", q_list)
    if len(q_list) > 1:
        qc.append(ZGate().control(len(q_list)-1), q_list)
    elif len(q_list) == 1:
        qc.z(q_list[0])
    
    for i in range(n):
        qc.x(i)
    
    qc.compose(qc_init, inplace=True)
    
    tqc = transpile(qc, basis_gates=basis_gates, optimization_level=3)
    for i in range(n):
        tqc.measure(i, i)
    # Run simulator
    iresult = ideal_simulator.run(tqc).result()
    icounts = iresult.get_counts(0)
    
    # Run simulator
    result = backend_sim.run(tqc).result()
    err = 0
    counts = result.get_counts(0)
    for key, count in counts.items():
        if key in icounts.keys():
            if icounts[key] > counts[key]:
                err += icounts[key]-counts[key]
    err_rate = err/shots
    print(f'    ideal searched state is[{max(icounts, key=icounts.get)}]')
    print(f'    noised searched state is[{max(counts, key=counts.get)}]')
    
    '''unitary = qi.Operator(qc_init).data
    abs_unitary = np.abs(unitary)'''
    '''abs_unitary = get_unitary_with_abs(qc_init)
    eigenvalues = np.linalg.eigvals(abs_unitary)'''
    
    return len(always_zero), err_rate, icounts, counts, qc_init, tqc

In [ ]:
num_search_list = []
num_targ_list = []
num_bitstring = []
err_list = []
reduced_list = []
ratio_search = []
init_qc = []
tqc = []
for n in range(5, 7):
    bitstrings = [f"{i:0{n}b}" for i in range(2**n)]
    for num_targ in range(1, 3):
        for num_search in range(num_targ*2+1, 4*n):
            bs_search = random.sample(bitstrings, num_search)
            err_list_iter = []
            reduced_list_iter = []
            for iter in range(30):
                res = err_grover(n, bs_search, random.sample(bs_search, num_targ))
                err_list.append(res[1])
                reduced_list.append(res[0])
                num_search_list.append(num_search)
                num_targ_list.append(num_targ)
                num_bitstring.append(2**n)
                ratio_search.append(num_search/2**n)
                init_qc.append(res[4])
                tqc.append(res[5])

In [ ]:
result = {'ratio_search': ratio_search, 'reduced_list': reduced_list, 'err_list': err_list, 'num_search_list': num_search_list, 'num_bitstring': num_bitstring}
with open(f'Noise_model_test3.json', 'w') as file:
    json.dump(result, file)